# This notebook performs activation of sugar GRNs + bitter GRNs while quantifying the effect on downstream neurons in the sugar pathway, and also tests the effect of silencing downstream bitter neurons.


## Code in this notebook is adapted from code from Philip Shiu (see https://github.com/philshiu/Drosophila_brain_model).

In [1]:
from model import run_exp
from model import default_params as params
import utils as utl
import pandas
from brian2 import Hz

config = {
    'path_res'  : './results/example',                              # directory to store results
    'path_comp' : './2023_03_23_completeness_630_final.csv',        # csv of the complete list of Flywire neurons
    'path_con'  : './2023_03_23_connectivity_630_final.parquet',    # connectivity data
    'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
}

INFO       Cache size for target 'cython': 1301231437 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]


## Underlying connectivity data
The connectivity of the fly brain is stored in the folowing files:
- neurons present: `config['path_comp']`
- connectivity between neurons: `config['path_con]`

## Model parameters
The equation and constants for the leaky integrate and fire model are defined 
in the dictionary `default_params` in the beginning of the file `model.py`:

```
default_params = {
    # trials
    't_run'     : 1000 * ms,              # duration of trial
    'n_run'     : 30,                     # number of runs

    'v_0'       : -52 * mV,               # resting potential
    'v_rst'     : -52 * mV,               # reset potential after spike
    [...]
```
We can also change values
and pass the modified dictionary to the model (see Experiment 1).

## Define neurons

In [2]:
# Define sugar-sensing neurons in left hemisphere 
# (same ones as Shiu paper but they called it right; brain flip was discovered later)

neu_sugar_L = [
    720575940624963786,
    720575940630233916,
    720575940637568838,
    720575940638202345,
    720575940617000768,
    720575940630797113,
    720575940632889389,
    720575940621754367,
    720575940621502051,
    720575940640649691,
    720575940639332736,
    720575940616885538,
    720575940639198653,
    720575940620900446,
    720575940617937543,
    720575940632425919,
    720575940633143833,
    720575940612670570,
    720575940628853239,
    720575940629176663,
    720575940611875570,
]

neu_bitter_L = [
    720575940602353632,
    720575940604027168,
    720575940610259370,
    720575940610481370,
    720575940610483162,
    720575940613061118,
    720575940614281266,
    720575940617094208,
    720575940618600651,
    720575940619028208,
    720575940619197093,
    720575940621008895,
    720575940621778381,
    720575940622298631,
    720575940626287336,
    720575940627578156,
    720575940627692048,
    720575940629146711,
    720575940630195909,
    720575940645743412,
    720575940646212996,
]


Define a mapping from the flywire IDs to custom names. The above neurons are called sugar_1, sugar_2 etc:

In [3]:
flyid2name = { f: 'sugar_L_{}'.format(i+1) for i, f in enumerate(neu_sugar_L) }
flyid2name

{720575940624963786: 'sugar_L_1',
 720575940630233916: 'sugar_L_2',
 720575940637568838: 'sugar_L_3',
 720575940638202345: 'sugar_L_4',
 720575940617000768: 'sugar_L_5',
 720575940630797113: 'sugar_L_6',
 720575940632889389: 'sugar_L_7',
 720575940621754367: 'sugar_L_8',
 720575940621502051: 'sugar_L_9',
 720575940640649691: 'sugar_L_10',
 720575940639332736: 'sugar_L_11',
 720575940616885538: 'sugar_L_12',
 720575940639198653: 'sugar_L_13',
 720575940620900446: 'sugar_L_14',
 720575940617937543: 'sugar_L_15',
 720575940632425919: 'sugar_L_16',
 720575940633143833: 'sugar_L_17',
 720575940612670570: 'sugar_L_18',
 720575940628853239: 'sugar_L_19',
 720575940629176663: 'sugar_L_20',
 720575940611875570: 'sugar_L_21'}

In [4]:
# Define other neurons

id_zorro_L = 720575940629888530
id_rattle_L = 720575940638103349
id_fmin_L = 720575940614763666
id_clavicle_L = 720575940655014049
id_usnea_L = 720575940632648612
id_phantom_L = 720575940616103218
id_g2n_L = 720575940620874757
id_fdg_L = 720575940631997032
id_bract1_L = 720575940645045527
id_bract2_L = 720575940610001220
id_roundup_L = 720575940623211725

id_foxglove_L = 720575940628755815
id_bluebell_L = 720575940631168057
id_mn9_L = 720575940645521262

id_zorro_R = 720575940611015122
id_rattle_R = 720575940630461660
id_fmin_R = 720575940636675214
id_clavicle_R = 720575940632648868
id_usnea_R = 720575940641366517
id_phantom_R = 720575940637763135
id_g2n_R = 720575940623718380
id_fdg_R = 720575940647030324
id_bract1_R = 720575940626557442
id_bract2_R = 720575940627285267
id_roundup_R = 720575940607272649

id_foxglove_R = 720575940611160930
id_bluebell_R = 720575940635166571
id_mn9_R = 720575940660219265


# walking-related neurons

id_p9_L = 720575940635872101
id_p9_R = 720575940627652358
id_odn1_L = 720575940626730883
id_odn1_R = 720575940620300308
id_cdn1_L = 720575940614730914
id_cdn1_R = 720575940612603289
id_dna01_L = 720575940611735514
id_dna01_R = 720575940623019544
id_dna02_L = 720575940629327659
id_dna02_R = 720575940604737708
id_dna06_L = 720575940624986919
id_dna06_R = 720575940646952324
id_dnb02_L = 720575940616936226
id_dnb02_R = 720575940646753412


# bitter pathway neurons

id_scapula_L = 720575940624600840
id_scapula_R = 720575940620062288

id_marge_L = 720575940610690372
id_marge_R = 720575940620262465 # this is putative, based on "similar neuron" search in flywire

id_mime_L = [
    720575940638557312,
    720575940637742150,
    720575940624253991,
    720575940630575943,
    720575940614223154,
    720575940634788247,
    720575940624273149
]

id_mime_R = [
    720575940622688765,
    720575940627936515,
    720575940637788137,
    720575940629598935,
    720575940621149709,
    720575940634844408
]


# Running simulations
## Background info:
To run a simulation exciting these nerons we have to call `run_exp` supplying the following:
- unique name for the simulation: `exp_name`
- a list of neurons we want to stimulate: `neu_sugar`
- the connectivity data: `config['path_comp']` and `config['path_con]`
- path to store the output: `config['path_res']`
- number of CPU cores use: `config['n_procs]`

The `.parquet` file created during a simulation contains all spikes events of all neurons in the model.
We load the data again from disk by passing a list of result files to the `utl.load_exps` function.

The spike times can be converted to spike rates [Hz] via `utl.get_rate`, which requires the duration of each trial.
`utl.get_rate` returns `pandas.DataFrame` objects:
1. spike rate for each neuron (rows) in each experiment (column): `df_rate`
2. standard deviation of rate across trials: `df_rate_std`

For convenience, we can optionally pass the `flyid2name` dictionary to `utl.get_rate` in order to convert flywire IDs into
meaningful names.

In [5]:
#show default params
params

{'t_run': 1. * second,
 'n_run': 30,
 'v_0': -52. * mvolt,
 'v_rst': -52. * mvolt,
 'v_th': -45. * mvolt,
 't_mbr': 20. * msecond,
 'tau': 5. * msecond,
 't_rfc': 2.2 * msecond,
 't_dly': 1.8 * msecond,
 'w_syn': 275. * uvolt,
 'r_poi': 150. * hertz,
 'r_poi2': 0. * hertz,
 'f_poi': 250,
 'eqs': '\ndv/dt = (v_0 - v + g) / t_mbr : volt (unless refractory)\ndg/dt = -g / tau               : volt (unless refractory) \nrfc                            : second\n',
 'eq_th': 'v > v_th',
 'eq_rst': 'v = v_rst; w = 0; g = 0 * mV'}

## Define a function to activate specified neurons and extract the firing rates from another list of neurons.

In [6]:
def run_simulation(params, neurons_to_activate, neurons_to_record, activation_names, record_names, prefix, ctrl_filename, savepath):

    #run experiment
    for id in neurons_to_activate: # go through each neuron
        ind = neurons_to_activate.index(id) # get index
        name = activation_names[ind] #
        prefix2 = prefix + '{}'
        run_exp(exp_name=prefix2.format(name), neu_exc=id, params=params, **config)
        
    # load activation data into a dataframe

    filelist = [ctrl_filename] # activating sugar neurons alone
    for id in neurons_to_activate: 
        ind = neurons_to_activate.index(id)
        name = activation_names[ind]
        datafilename = './results/example/' + prefix + name + '.parquet'
        filelist.append(datafilename)
    
    df_spike = utl.load_exps(filelist)
    df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)


    # get dataframe of rates for all neurons and all conditions

    rates = []
    rates_std = []

    for neuron in neurons_to_record:
        rates.append(df_rate.loc[neuron])
        rates_std.append(df_rate_std.loc[neuron])
    
    rates = pandas.DataFrame(rates) # convert to dataframe
    rates_std = pandas.DataFrame(rates_std) # convert to dataframe

    rates = rates.assign(name=record_names)
    rates_std = rates_std.assign(name=record_names)
    
    # save dataframes to csv
    df_rate.fillna(0).to_csv(savepath + prefix + 'allrates.csv')
    df_rate_std.fillna(0).to_csv(savepath + prefix + 'allrates_std.csv')
    rates.to_csv(savepath + prefix + 'rates.csv')
    rates_std.to_csv(savepath + prefix + 'rates_std.csv')

    return df_rate, df_rate_std, rates, rates_std

## Run simulations multiple times at diff frequencies, while stimulating combinations of sugar pathway neurons bilaterally.

In [7]:
# list of neuron ids - this is a reference for all expts

ids_2N_L = [id_zorro_L, id_rattle_L, id_fmin_L, id_clavicle_L, id_usnea_L, id_phantom_L, id_g2n_L]
ids_2N_R = [id_zorro_R, id_rattle_R, id_fmin_R, id_clavicle_R, id_usnea_R, id_phantom_R, id_g2n_R]
names_2N = ['zorro','rattle','fmin','clavicle','usnea','phantom','g2n']

ids_3N_L = [id_fdg_L, id_bract1_L, id_bract2_L]
ids_3N_R = [id_fdg_R, id_bract1_R, id_bract2_R]
names_3N = ['fdg','bract1','bract2']

ids_4N_L = [id_roundup_L, id_foxglove_L, id_bluebell_L]
ids_4N_R = [id_roundup_R, id_foxglove_R, id_bluebell_R]
names_4N = ['roundup','foxglove','bluebell']

motor_ids_L = [id_mn9_L]
motor_ids_R = [id_mn9_R]
names_motor = ['MN9']

# list of neurons to record in all expts - here only recording neurons on left side
neurons_to_record = [id_zorro_L, id_rattle_L, id_fmin_L, id_clavicle_L, id_usnea_L, id_phantom_L, id_g2n_L, id_fdg_L, id_bract1_L,id_bract2_L, id_roundup_L, id_foxglove_L, id_bluebell_L, id_mn9_L]
record_names = ['zorro','rattle','fmin','clavicle','usnea','phantom','g2n','fdg','bract1','bract2','roundup','foxglove','bluebell','MN9']



## Run simulation: first just activate sugar and bitter neurons together.

In [8]:
# define neurons to activate

neurons_to_activate = [neu_sugar_L + neu_bitter_L]

activation_names = ['sugar+bitterGRNs']

In [9]:
# Set params and run simulation

for stim_rate in [50, 100, 200]: 
    
    params['r_poi'] = stim_rate * Hz
    prefix = 'sugarbitter_activ_' + str(stim_rate) + 'Hz_'
    ctrl_filename = './results/example/sugarR_' + str(stim_rate) + 'Hz.parquet' # this is sugarGRN activation only
    savepath = 'results/sugarbitter_activ ' + str(stim_rate) + 'Hz/'
    
    run_simulation(params, neurons_to_activate, neurons_to_record, activation_names, record_names, prefix, ctrl_filename, savepath)    

>>> Skipping experiment sugarbitter_activ_50Hz_sugar+bitterGRNs because results/example/sugarbitter_activ_50Hz_sugar+bitterGRNs.parquet exists and force_overwrite = False
>>> Skipping experiment sugarbitter_activ_100Hz_sugar+bitterGRNs because results/example/sugarbitter_activ_100Hz_sugar+bitterGRNs.parquet exists and force_overwrite = False
>>> Skipping experiment sugarbitter_activ_200Hz_sugar+bitterGRNs because results/example/sugarbitter_activ_200Hz_sugar+bitterGRNs.parquet exists and force_overwrite = False


## Now try silencing potential downstream bitter neurons to see if the bitter suppression is affected.

In [10]:
# Function for silencing - currently only works with activating one set of neurons at a time

def run_simulation_silencing(params, neurons_to_activate, neurons_to_silence, neurons_to_record, silencing_names, record_names, prefix, ctrl_filename, savepath):

    #run experiment
    for id in neurons_to_silence: # go through each neuron
        ind = neurons_to_silence.index(id) # get index
        name = silencing_names[ind] #
        prefix2 = prefix + '{}'
        run_exp(exp_name=prefix2.format(name), neu_exc=neurons_to_activate, neu_slnc=id, params=params, **config)
        
    # load data into a dataframe

    filelist = [ctrl_filename,ctrl_filename2] # activating sugar GRNs or sugar+bitter GRNs without silencing
    for id in neurons_to_silence: 
        ind = neurons_to_silence.index(id)
        name = silencing_names[ind]
        datafilename = './results/example/' + prefix + name + '.parquet'
        filelist.append(datafilename)
    
    df_spike = utl.load_exps(filelist)
    df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)


    # get dataframe of rates for all neurons and all conditions

    rates = []
    rates_std = []

    for neuron in neurons_to_record:
        rates.append(df_rate.loc[neuron])
        rates_std.append(df_rate_std.loc[neuron])
    
    rates = pandas.DataFrame(rates) # convert to dataframe
    rates_std = pandas.DataFrame(rates_std) # convert to dataframe

    rates = rates.assign(name=record_names)
    rates_std = rates_std.assign(name=record_names)
    
    # save dataframes to csv
    df_rate.fillna(0).to_csv(savepath + prefix + 'allrates.csv')
    df_rate_std.fillna(0).to_csv(savepath + prefix + 'allrates_std.csv')
    rates.to_csv(savepath + prefix + 'rates.csv')
    rates_std.to_csv(savepath + prefix + 'rates_std.csv')

    return df_rate, df_rate_std, rates, rates_std

In [11]:
silencing_names = ['scapula','marge','mime']

neurons_to_silence = [
    [id_scapula_L] + [id_scapula_R],
    [id_marge_L] + [id_marge_R],
    id_mime_L + id_mime_R
]

In [12]:
# Set params and run simulation

neurons_to_activate = neu_sugar_L + neu_bitter_L

for stim_rate in [50, 100, 200]: 
    
    params['r_poi'] = stim_rate * Hz
    prefix = 'sugarbitter_slc_' + str(stim_rate) + 'Hz_'
    ctrl_filename = './results/example/sugarR_' + str(stim_rate) + 'Hz.parquet' # this is sugar GRN activation only
    ctrl_filename2 = './results/example/sugarbitter_activ_' + str(stim_rate) + 'Hz_sugar+bitterGRNs.parquet' # this is sugar+bitter GRN activation only
    savepath = 'results/' + str(stim_rate) + 'Hz bilat silencing/'
    savepath = 'results/sugarbitter_activ_slc ' + str(stim_rate) + 'Hz/'
    
    run_simulation_silencing(params, neurons_to_activate, neurons_to_silence, neurons_to_record, silencing_names, record_names, prefix, ctrl_filename, savepath)    

>>> Skipping experiment sugarbitter_slc_50Hz_scapula because results/example/sugarbitter_slc_50Hz_scapula.parquet exists and force_overwrite = False
>>> Skipping experiment sugarbitter_slc_50Hz_marge because results/example/sugarbitter_slc_50Hz_marge.parquet exists and force_overwrite = False
>>> Skipping experiment sugarbitter_slc_50Hz_mime because results/example/sugarbitter_slc_50Hz_mime.parquet exists and force_overwrite = False
>>> Skipping experiment sugarbitter_slc_100Hz_scapula because results/example/sugarbitter_slc_100Hz_scapula.parquet exists and force_overwrite = False
>>> Skipping experiment sugarbitter_slc_100Hz_marge because results/example/sugarbitter_slc_100Hz_marge.parquet exists and force_overwrite = False
>>> Skipping experiment sugarbitter_slc_100Hz_mime because results/example/sugarbitter_slc_100Hz_mime.parquet exists and force_overwrite = False
>>> Skipping experiment sugarbitter_slc_200Hz_scapula because results/example/sugarbitter_slc_200Hz_scapula.parquet exis